# Getting the Agent Job Steps into an Excel Sheet

This Notebook will connect to an instance and create a colour coded excel sheet of the Agent Jobs showing

- last run time
- outcome
- on success and on failure actions of each step 
- last duration
- schedules
- job enabled and schedule enabled 
- Next run time




You can check if you have the modules and install them in your user scope with this block

In [ ]:
#Check for modules and install
$Modules = 'dbatools', 'DbaChecks', 'ImportExcel'

if((Get-PsRepository -Name PSGallery).InstallationPolicy -ne 'Trusted'){
    Write-Output "             #######################################################"
    Write-Output "The PowerShell Gallery is not trusted so I will trust it so that we can install the modules without interaction"
    Write-Output "             ######     Please follow the Setting Up notebook ######"
} else {
    Write-Output "The PowerShell Gallery is trusted I will continue"
}
$Modules.ForEach{
    If(-not(Get-Module $psitem -ListAvailable)){
        Write-Output "             #######################################################"
        Write-Output "                        We don't have the $psitem module."
        Write-Output "             ######     Please follow the Setting Up notebook ######"
    }
    else {
        Write-Output "We have the $psitem module already"
    }
}

In [ ]:
$ExcelDirectory = 'c:\temp\' # Alter this to the directory you want the file created
$SqlInstances = 'SQL2017N5' # Alter this for the SQL Instance you want to get the agent jobs for
$FilterInclude = '' #'OLA*','DevOps*', 'SSIS*' # to filter to include specific jobs add them here in single quotes use * for the wild card IE '*FULL*', 'SQLWATCH*' , 'DBA*'
$FilterExclude = '' # 'DevOps*', 'SSIS*' # to filter to exclude specific jobs add them here in single quotes use * for the wild card IE '*FULL*', 'SQLWATCH*' , 'DBA*'
Write-Output "Processing $sqlinstances"

In [ ]:
# The code is below - just click the play button to the left - it will take a minute or so and you will see it spinning
$ExcelFile = $ExcelDirectory + '\AgentJob_Check_' + (Get-Date).ToString('yyyy-MM-dd') + '.xlsx'

Write-Output "    FileName is $ExcelFile"

$WorkSheetName = "Agent Jobs"
$JobName = @{Name = 'Job Name' ; Expression = {$Psitem.AgentJob}}
$Isenabled = @{Name = 'Job Enabled' ; Expression = {$Psitem.Parent.IsEnabled}}
$JobScheduleName =  @{Name = 'Schedule' ; Expression = {@($Psitem.Parent.JobSchedules).ForEach{$_.Name} -join ' ' }}
$JobScheduleEnabled =  @{Name = 'Schedule Enabled' ; Expression = {@($Psitem.Parent.JobSchedules).ForEach{$_.IsEnabled} -join ' '}}
$NextRunDate =  @{Name = 'Next Run Date' ; Expression = {$Psitem.Parent.NextRunDate}}
$LastRunDate =  @{Name = 'Last Run Date' ; Expression = {if($Psitem.Parent.LastRunDate -eq '01/01/0001 00:00:00'){'NULL'}Else {$Psitem.Parent.LastRunDate} }}
$LastRunOutCome =  @{Name = 'Last RunOutCome' ; Expression = {$Psitem.Parent.LastRunOutCome}}
$selectObjectSplat = @{
    Property = 'SqlInstance', $LastRunDate , $JobName, 'Name', $LastRunOutCome , 'OnSuccessAction', 'OnFailAction', 'LastRunDuration', $IsEnabled, $JobScheduleName, $JobScheduleEnabled, $NextRunDate
}

if (($FilterInclude -eq '') -and ($FilterExclude -eq '') ){
    $WhereString = {$psitem.AgentJob -like '*'}
} elseif (($FilterInclude -ne '') -and ($FilterExclude -eq '')) {       
$WhereArray = @()  
    $FilterInclude.ForEach{
        $WhereArray  += '$psitem.AgentJob -like ''' + $_ + ''''
    }         
    $WhereString = $WhereArray -Join " -or "   
} elseif (($FilterInclude -eq '') -and ($FilterExclude -ne '')) {
    $WhereArray = @()  
    $FilterExclude.ForEach{
        $WhereArray  += '$psitem.AgentJob -notlike ''' + $_ + ''''
    }         
    $WhereString = $WhereArray -Join " -or "   
} else{
    $WhereIncArray = @()  
    $FilterInclude.ForEach{
        $WhereIncArray  += '$psitem.AgentJob -like ''' + $_ + ''''
    } 
    $WhereIncString = $WhereIncArray -Join " -or " 
    $WhereExcArray = @()  
    $FilterExclude.ForEach{
        $WhereExcArray  += '$psitem.AgentJob -notlike ''' + $_ + ''''
    }         
    $WhereExcString = $WhereExcArray -Join " -and " 
    $WhereString = $WhereExcString = "(" + $WhereIncString + ") -and " + $WhereExcString   
}

$whereObjectSplat = @{
    FilterScript = [scriptblock]::Create( $WhereString )
}

Get-DbaAgentJobStep -SqlInstance $SqlInstances | Where-Object @whereObjectSplat  | Select-Object @selectObjectSplat  | Export-Excel -Path $ExcelFile -WorksheetName $WorkSheetName -AutoSize -FreezeTopRow -AutoFilter

$Excel = Open-ExcelPackage -Path $ExcelFile
$WorkSheet = $excel.Workbook.Worksheets[$WorkSheetName] 

$addConditionalFormattingSplat = @{
    StopIfTrue = $true
    RuleType = 'ContainsText'
    BackgroundColor = 'LightGray' #'PowderBlue'
    Worksheet = $worksheet
    Address = "E2:E1048576"
    ConditionValue = 'Unknown'
    Bold = $true
}
Add-ConditionalFormatting @addConditionalFormattingSplat 

$addConditionalFormattingSplat = @{
    StopIfTrue = $true
    RuleType = 'ContainsText'
    BackgroundColor = 'Green'
    Worksheet = $worksheet
    Address = "E2:E1048576"
    ConditionValue = 'Succeeded'
    Bold = $true
}
Add-ConditionalFormatting @addConditionalFormattingSplat

$addConditionalFormattingSplat = @{
    StopIfTrue = $true
    RuleType = 'ContainsText'
    BackgroundColor = 'Red'
    Worksheet = $worksheet
    Address = "E2:E1048576"
    ConditionValue = 'Failed'
    Bold = $true
}
Add-ConditionalFormatting @addConditionalFormattingSplat

$addConditionalFormattingSplat = @{
    StopIfTrue = $true
    RuleType = 'ContainsText'
    BackgroundColor = 'PowderBlue'
    Worksheet = $worksheet
    Address = "E2:E1048576"
    ConditionValue = 'Unknown'
    Bold = $true
}
Add-ConditionalFormatting @addConditionalFormattingSplat 

$addConditionalFormattingSplat = @{
    StopIfTrue = $true
    RuleType = 'ContainsText'
    BackgroundColor = 'PowderBlue'
    Worksheet = $worksheet
    Address = "F2:F1048576"
    ConditionValue = 'QuitWithSuccess'
    Bold = $true
}
Add-ConditionalFormatting @addConditionalFormattingSplat

$addConditionalFormattingSplat = @{
    StopIfTrue = $true
    RuleType = 'ContainsText'
    BackgroundColor = 'GoldenRod'
    Worksheet = $worksheet
    Address = "F2:F1048576"
    ConditionValue = 'GoToNextStep'
    Bold = $true
}
Add-ConditionalFormatting @addConditionalFormattingSplat

$addConditionalFormattingSplat = @{
    StopIfTrue = $true
    RuleType = 'ContainsText'
    BackgroundColor = 'PeachPuff' # 'BurlyWood'
    Worksheet = $worksheet
    Address = "F2:F1048576"
    ConditionValue = 'GoToStep'
    Bold = $true
}
Add-ConditionalFormatting @addConditionalFormattingSplat

$addConditionalFormattingSplat = @{
    StopIfTrue = $true
    RuleType = 'ContainsText'
    BackgroundColor = 'PowderBlue'
    Worksheet = $worksheet
    Address = "G2:G1048576"
    ConditionValue = 'QuitWithFailure'
    Bold = $true
}
Add-ConditionalFormatting @addConditionalFormattingSplat

$addConditionalFormattingSplat = @{
    StopIfTrue = $true
    RuleType = 'ContainsText'
    BackgroundColor = 'GoldenRod'
    Worksheet = $worksheet
    Address = "G2:G1048576"
    ConditionValue = 'GoToNextStep'
    Bold = $true
}
Add-ConditionalFormatting @addConditionalFormattingSplat

$addConditionalFormattingSplat = @{
    StopIfTrue = $true
    RuleType = 'ContainsText'
    BackgroundColor = 'PeachPuff' # 'BurlyWood'
    Worksheet = $worksheet
    Address = "G2:G1048576"
    ConditionValue = 'GoToStep'
    Bold = $true
}
Add-ConditionalFormatting @addConditionalFormattingSplat

$addConditionalFormattingSplat = @{
    StopIfTrue = $true
    RuleType = 'ContainsText'
    BackgroundColor = 'Red'
    Worksheet = $worksheet
    Address = "G2:G1048576"
    ConditionValue = 'QuitWithSuccess'
    Bold = $true
}
Add-ConditionalFormatting @addConditionalFormattingSplat

$addConditionalFormattingSplat = @{
    StopIfTrue = $true
    RuleType = 'ContainsText'
    BackgroundColor = 'Red'
    Worksheet = $worksheet
    Address = "I2:I1048576"
    ConditionValue = 'FALSE'
    Bold = $true
}
Add-ConditionalFormatting @addConditionalFormattingSplat

$addConditionalFormattingSplat = @{
    StopIfTrue = $true
    RuleType = 'ContainsText'
    BackgroundColor = 'PaleGreen'
    Worksheet = $worksheet
    Address = "I2:I1048576"
    ConditionValue = 'TRUE'
    Bold = $true
}
Add-ConditionalFormatting @addConditionalFormattingSplat

$addConditionalFormattingSplat = @{
    StopIfTrue = $true
    RuleType = 'ContainsText'
    BackgroundColor = 'PaleGreen'
    Worksheet = $worksheet
    Address = "K2:K1048576"
    ConditionValue = 'TRUE'
    Bold = $true
}
Add-ConditionalFormatting @addConditionalFormattingSplat


$addConditionalFormattingSplat = @{
    StopIfTrue = $true
    RuleType = 'ContainsText'
    BackgroundColor = 'Red'
    Worksheet = $worksheet
    Address = "K2:K1048576"
    ConditionValue = 'FALSE'
    Bold = $true
}
Add-ConditionalFormatting @addConditionalFormattingSplat

Close-ExcelPackage $excel

Invoke-Item $ExcelFile